In [20]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [21]:
def preprocess_text(text: str):
    text = text.strip()
    try:
        if text[-1].isdigit():
            text = text[:-1] + " " + text[-1]
    except IndexError:
        return np.nan
    text = text.lower().split("\n")
    while "" in text:
        text.remove("")
    for i in range(len(text)):
        text[i] = text[i].translate(str.maketrans('', '', '!"“”#$%&\'()*+/:;<=>?@[\\]^_`{|}~’')).strip()
        if text[i].startswith(","):
            text[i] = text[i][1:].strip()
        if text[i].startswith("."):
            text[i] = text[i][1:].strip()
        
        to_remove = []
        for j in range(len(text[i]) - 1):
            if text[i][j] == "," or text[i][j] == ".":
                if text[i][j + 1] == "," or text[i][j + 1] == '.':
                    to_remove.append(j + 1)
        for idx in to_remove[::-1]:
            text[i] = text[i][:idx] + text[i][idx+1:]

        add_to_start = lambda s: " " + s if not s.startswith(" ") else s
        add_to_end = lambda s: s + " " if not s.endswith(" ") else s
        space_out = lambda s, s2: s2.join([add_to_end(add_to_start(sentence)) for sentence in s.split(s2)]).strip()
        to_space_out = [".", ",", "-"]
        for c in to_space_out:
            text[i] = space_out(text[i], c)
    return " ".join(text)

In [22]:
df = pd.read_csv("wikihow.csv", names=["summary", "title", "text"])
df = df.drop([0], axis=0).dropna()

In [23]:
for idx in tqdm(range(len(df)), total=len(df), leave=True, position=0):
    for j in range(3):
        df.iloc[idx][j] = preprocess_text(df.iloc[idx][j])

100%|██████████| 214294/214294 [02:20<00:00, 1522.62it/s]


In [26]:
df = df.dropna()

In [30]:
df.to_csv("wikihow-cleaned.csv", index=False)